In [1]:
import sys
sys.path.append('../network')
import network
import svg_draw as sd
import pandas as pd
from reportlab.graphics import renderPDF
from svglib.svglib import svg2rlg
import numpy as np
import copy
import os

def persentage_cutoff(net_df, top=0.2):
    upper = np.triu(net_df.values, k=1)
    values = upper.flatten()
    values = values[values != 0]
    threshold = np.quantile(values, 1-top)
    return threshold

def draw_network(fr, size_series, layout_df, max_edge_width = 4, max_r = 10, min_width = 1, cutoff=0.9, edge_cluster=None):
    origin = (530, 530)
    common_taxon = list(set(fr.index).intersection(set(layout_df.index)))
    fr = fr.loc[common_taxon, common_taxon]
    circles = ''
    pr_max = size_series.max()
    pr_range = pr_max
    xy_df = pd.DataFrame()
    for idx in common_taxon:
        theta = layout_df.loc[idx, 'theta']
        r = layout_df.loc[idx, 'r']
        color = layout_df.loc[idx, 'color']
        x, y = network.convert(theta, r, origin)
        xy_df.loc[idx, 'x'] = x
        xy_df.loc[idx, 'y'] = y
        
        swidth = 1
        pr = size_series[idx]
        r = max_r*pr/pr_range
        circle = sd.draw_circle(x, y, swidth=swidth, r=r, fill=color, id=idx, style=True)
        circles +=  circle
    
    paths = ""
    ntaxon = fr.shape[0]
    taxon = list(fr.index)
    for i in range(ntaxon):
        taxa1 = taxon[i]
        x1 = xy_df.loc[taxa1, 'x']
        y1 = xy_df.loc[taxa1, 'y']
        for j in range(i+1, ntaxon):
            taxa2 = taxon[j]
            fr_value = fr.loc[taxa1, taxa2]
            if fr_value > cutoff:
                x2 = xy_df.loc[taxa2, 'x']
                y2 = xy_df.loc[taxa2, 'y']
                id = "{}_{}".format(taxa1, taxa2)
                x3, y3 = network.compute_convex((x1, y1), (x2, y2))
                edge_width = max_edge_width*(fr_value-cutoff)/(fr.max().max()-cutoff)
                if edge_cluster is not None:
                    if edge_cluster.loc[taxa1, 'cluster'] == edge_cluster.loc[taxa2, 'cluster']:
                        path = sd.draw_curve(x1, y1, x3, y3, x2, y2, swidth=edge_width, id=id, style=True)
                    else:
                        path = sd.draw_curve(x1, y1, x3, y3, x2, y2, stroke='red', swidth=edge_width, id=id, style=True)
                else:
                    path = sd.draw_curve(x1, y1, x3, y3, x2, y2, swidth=edge_width, id=id, style=True)
                paths += path
    #title = '<text x="15" y="40" id="sp">{}</text>'.format(eigen)
    #return title + paths + circles
    return paths + circles




In [2]:
legend = '<circle cx="1090" cy="20" r="12" stroke="black" stroke-width="1" fill="#D95F02" /> \n \
<text x="1110" y="25" class="legend">Alphaproteobacteria</text>\n \
<circle cx="1090" cy="50" r="12" stroke="black" stroke-width="1" fill="#85D696" />\n \
<text x="1110" y="55" class="legend">Gammaproteobacteria</text>\n \
<circle cx="1090" cy="80" r="12" stroke="black" stroke-width="1" fill="#7EC9CE" />\n \
<text x="1110" y="85" class="legend">Clostridia</text>\n \
<circle cx="1090" cy="110" r="12" stroke="black" stroke-width="1" fill="#FFC68A" />\n \
<text x="1110" y="115" class="legend">Bacilli</text>\n \
<circle cx="1090" cy="140" r="12" stroke="black" stroke-width="1" fill="#999FDB" />\n \
<text x="1110" y="145" class="legend">Bacteroidia</text>\n \
<circle cx="1090" cy="170" r="12" stroke="black" stroke-width="1" fill="#C0E481" />\n \
<text x="1110" y="175" class="legend">Cyanobacteriia</text>\n \
<circle cx="1090" cy="200" r="12" stroke="black" stroke-width="1" fill="#CC99DB" />\n \
<text x="1110" y="205" class="legend">Actinomycetia</text>\n \
<circle cx="1090" cy="230" r="12" stroke="black" stroke-width="1" fill="#AAAAAA" />\n \
<text x="1110" y="235" class="legend">Other</text>'

def draw(content, legend):
    width = 1300
    height = 1200
    style = "\
        text {\n \
            font-family: Arial;\n\
        }\n\
        .legend {\n\
            font-size: 20px;\n\
        }\n\
        #sp {\n\
            font-size: 22px;\n\
            font-weight: bold;\n\
        }\n"
    
    return sd.canvas(width, height, content+legend, style)



In [3]:
outdir = '../result/S1_plasmid_net/net_plot'
feature_dir = '../result/S1_plasmid_net/feature'
if not os.path.exists(outdir):
    os.makedirs(outdir)
edge_cluster = pd.read_csv('../draw_network/class_genus.tsv', sep='\t', header=0, index_col=0)
edge_cluster = edge_cluster.rename(columns={'c': 'cluster'})
layout_df = pd.read_csv('../draw_network/sector_sp_layout.tsv', sep='\t', header=0, index_col=0)
fr_df = pd.read_csv(os.path.join(feature_dir, 'count_net.tsv'), sep='\t', header=0, index_col=0)
np.fill_diagonal(fr_df.values, 0)
tmp = copy.deepcopy(fr_df)
tmp[tmp>0] = 1
degree = tmp.sum(axis=1).apply(lambda x: np.log2(x+1))
fr_df_log = fr_df.applymap(lambda x: np.log2(x+1))
cutoff = persentage_cutoff(fr_df_log, top=0.30)
s_sector= draw(draw_network(fr_df_log, degree, layout_df, cutoff=cutoff, edge_cluster=edge_cluster), legend)
svg_dir = os.path.join(outdir, 'degree.svg')
with open(svg_dir, 'w') as fp:
    fp.write(s_sector)

#drawing = svg2rlg(svg_dir)
#pdf_path = 'test_plot.pdf'
#renderPDF.drawToFile(drawing, pdf_path)
            

In [4]:
betweenness = pd.read_csv(os.path.join(feature_dir, 'betweeness.tsv'), sep='\t', header=0, index_col=0)['betweenness'].apply(lambda x: np.log2(x+1))
s_sector= draw(draw_network(fr_df_log, betweenness, layout_df, cutoff=cutoff, edge_cluster=edge_cluster), legend)
svg_dir = os.path.join(outdir, 'betweenness.svg')
with open(svg_dir, 'w') as fp:
    fp.write(s_sector)

In [5]:
pr = pd.read_csv(os.path.join(feature_dir, 'pagerank.tsv'), sep='\t', header=0, index_col=0)['pr'].apply(lambda x: np.log2(x+1))
s_sector= draw(draw_network(fr_df_log, pr, layout_df, cutoff=cutoff, edge_cluster=edge_cluster), legend)
svg_dir = os.path.join(outdir, 'pagerank.svg')
with open(svg_dir, 'w') as fp:
    fp.write(s_sector)

In [6]:
# connected component
cc = {}
i = 0
with open(os.path.join(feature_dir, 'connected_component.tsv')) as fp:
    for line in fp.readlines():
        genera = line.strip().split(',')
        num = len(genera)
        if num > 1:
            cc[i] = {}
            cc[i]['genera'] = copy.deepcopy(genera)
            cc[i]['num'] = num
            i += 1
take_list = []
for i in cc:
    if cc[i]['num'] < 11:
        take_list += cc[i]['genera']
take_df = fr_df.loc[take_list, take_list]
tmp = copy.deepcopy(take_df)
tmp[tmp>0] = 1
degree = tmp.sum(axis=1).apply(lambda x: np.log2(x+1))
edge_df = take_df.applymap(lambda x: np.log2(x+1))
s_sector= draw(draw_network(edge_df, degree, layout_df, cutoff=0, edge_cluster=edge_cluster), legend)
svg_dir = os.path.join(outdir, 'connected_component.svg')
with open(svg_dir, 'w') as fp:
    fp.write(s_sector)